In [2]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [3]:
def create_server_connection(host_name,user_name,pwd):
    connection=None
    try:
        connection=mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=pwd
        )
        print("MYSQL Database connection successful")
    except Error as err:
        print(f"Error : {err}")
    return connection


host_name='localhost'
user_name='root'             
pwd='12345'
db_name='sql_demo1'
        
connection=create_server_connection(host_name,user_name,pwd)  

MYSQL Database connection successful


In [4]:
def create_database(connection,query):
    cursor=connection.cursor()
    try:
        cursor.execute(query)
        print("Database create successfully")
    except Error as err:
        print(f"Error : {err}")

              
create_db_query=f"""create database {db_name}
"""
create_database(connection,create_db_query)

Error : 1007 (HY000): Can't create database 'sql_demo1'; database exists


In [5]:
def create_db_connection(host_name,user_name,pwd,db_name):
    connection=None
    try:
        connection=mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=pwd,
            database=db_name
        )
        print("MYSQL Database connection successful")
    except Error as err:
        print(f"Error : {err}")
    return connection

In [6]:
def execute_query(connection,query):
    cursor=connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query Execute Successfully")
    except Error as err:
        print(f"Error : {err}")

In [7]:
create_countrylist_query="""
create table tbl_country_list(
country varchar(30)primary key not null);
"""

create_headerlist_query="""
create table tbl_header(
customer_name varchar(255),
last_consulted_date date,
customer_id varchar(18)
);
"""

create_trailerlist_query="""
create table tbl_trailer(
id varchar(5)
);
"""

connection=create_db_connection(host_name,user_name,pwd,db_name)

execute_query(connection,create_countrylist_query)
execute_query(connection,create_headerlist_query)
execute_query(connection,create_trailerlist_query)

MYSQL Database connection successful
Error : 1050 (42S01): Table 'tbl_country_list' already exists
Error : 1050 (42S01): Table 'tbl_header' already exists
Error : 1050 (42S01): Table 'tbl_trailer' already exists


In [8]:
def insert_countrylist(cntry_name):

    connection=create_db_connection(host_name,user_name,pwd,db_name)
    cursor=connection.cursor()
    
    insert_countrylist_query="INSERT INTO tbl_country_list (country) VALUES (%s)"
        
    val=list([cntry_name])

    try:        
        cursor.execute(insert_countrylist_query,val)
        connection.commit()

        print("Data Insert Successfully")
    except Error as err:
        print(f"Error : {err}")

In [9]:
def read_query(connection,query):
    cursor=connection.cursor()
    try:
        cursor.execute(query)
        results=cursor.fetchall()
        return results
    except Error as err:
        print(f"Error : {err}")

In [10]:
get_countrylist_query="""
select * from tbl_country_list
"""

connection=create_db_connection(host_name,user_name,pwd,db_name)
results=read_query(connection,get_countrylist_query)

for result in results:
    print(result)

MYSQL Database connection successful
('au',)
('usa',)
('ind',)
('phil',)


In [11]:
def create_countrywise_table(table_name):
    
    create_countrywisetable_query=f"""
        CREATE TABLE {table_name}(
        customer_name varchar(255) primary key not null,
        customer_id varchar(18) not null,
        customer_opendate date,
        last_consulted_date date,
        vaccination_type char(5),
        doctor_consulted varchar(255),
        state char(5),
        country char(30) not null,
        post_code int(5) null,
        date_of_birth date,
        active_customer char(1));
    """
    
    try:        

        connection=create_db_connection(host_name,user_name,pwd,db_name)
        execute_query(connection,create_countrywisetable_query)
        connection.commit()

        print("Country Table Created Successfully")
    except Error as err:
        print(f"Error : {err}")


In [20]:
#insert data into counntrywise table
import datetime


header_list=list([])
record_list=list([])

date_format = "%Y%m%d"
#dt_object = datetime.datetime.strptime(dt_string, date_format)

def input_records():
    connection=create_db_connection(host_name,user_name,pwd,db_name)
    cursor=connection.cursor()


    with open('demo_data.txt','r') as f:
        for line in f:
                line = line.strip()
                    
                if(line[1]=="T"):
                    for word in line.split("|"):
                        if(word!="T" and word!=" " and len(word)>0):
                            insert_trailer_query="INSERT INTO tbl_trailer (id) VALUES (%s)"
                            val=list([word])

                            cursor.execute(insert_trailer_query,val)
                            connection.commit()
                
                elif(line[1]=="H"):
                    for word in line.split("|"):
                        if(word!="H" and word!=" " and len(word)>0):
                            print(word)
                            header_list.append(word)

                    print(header_list)
                    insert_header_query="INSERT INTO tbl_header VALUES (%s,%s,%s)"
                    cursor.execute(insert_header_query,header_list)
                    connection.commit()
                    header_list.clear()
                
                elif(line[1]=="D"):
                    tbl_name=""
                    for word in line.split("|"):
                        if(word!="D" and word!=" " and len(word)>0):
                            print(word)
                            record_list.append(word)
                    
                    print(record_list)
                    
                    #read all country list
                    get_countrylist_query="""
                    select country from tbl_country_list
                    """
                    results=read_query(connection,get_countrylist_query)

                    isCountry_Found=False

                    for result in results:
                        result=','.join(result)
                        if((result.casefold())==(record_list[7].casefold())):
                            isCountry_Found=True        
                            break
                        else:
                            isCountry_Found=False

                    tbl_name=("tbl_"+record_list[7].casefold()).format()
                    print(tbl_name,type(tbl_name))
                    
                    if(isCountry_Found==False):
                        insert_countrylist(record_list[7].casefold())
                        create_countrywise_table(tbl_name)
                        
                        
                    record_list[9]=datetime.datetime.strptime(record_list[9], "%d%m%Y").strftime(date_format)

#                     insert_countrywise_records_query=f"""
#                      INSERT INTO {tbl_name}
#                              (customer_name,customer_id,customer_opendate,last_consulted_date,
#                              vaccination_type,doctor_consulted,state,
#                              country,post_code,date_of_birth,active_customer)
#                              VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

                    insert_countrywise_records_query=f"INSERT INTO {tbl_name} VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"


                    cursor.execute(insert_countrywise_records_query,record_list)
                    connection.commit()
 
                    record_list.clear()
                
        connection.close()      
#end of method

input_records()

MYSQL Database connection successful
Raj
123499
20111012
20131013
MVD
Mehta
MUNCH
Germany
12341
01031987
I
['Raj', '123499', '20111012', '20131013', 'MVD', 'Mehta', 'MUNCH', 'Germany', '12341', '01031987', 'I']
tbl_germany <class 'str'>


IntegrityError: 1062 (23000): Duplicate entry 'Raj' for key 'tbl_germany.PRIMARY'

In [19]:
cntry=input("Enter Country : ")

tbl_name="tbl_"+cntry

get_countrylist_query=f"""
select * from {tbl_name}
"""

connection=create_db_connection(host_name,user_name,pwd,db_name)
results=read_query(connection,get_countrylist_query)

for result in results:
    print(result)

Enter Country : ind
MYSQL Database connection successful
('Jay', '12345', datetime.date(2010, 10, 12), datetime.date(2012, 10, 13), 'MVD', 'Paul', 'BOS', 'IND', 66542, datetime.date(1987, 3, 6), 'A')
('John', '123456', datetime.date(2010, 10, 12), datetime.date(2012, 10, 13), 'MVD', 'Paul', 'NSW', 'IND', 38732, datetime.date(1987, 3, 6), 'A')
('Parth', '123458', datetime.date(2010, 10, 12), datetime.date(2012, 10, 13), 'MVD', 'Paul', 'TN', 'IND', 23313, datetime.date(1987, 3, 6), 'A')
